<a href="https://colab.research.google.com/github/omermehboob/NIDS-using-machine-learning-and-deep-learning/blob/main/DNN_1(NIDS_BINARY_7_features).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.utils.np_utils import to_categorical
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import Normalizer
import h5py
from keras import callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

In [2]:
df = pd.read_csv('/content/drive/MyDrive/Refined_data_set(label_encoding_18_features).csv')

In [3]:
df.shape

(1336498, 20)

In [4]:
df.head()

,Unnamed: 0,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Duration,Flow IAT Max,Flow IAT Mean,Flow IAT Min,Flow IAT Std,Fwd IAT Total,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Total Backward Packets,Total Fwd Packets,Total Length of Bwd Packets,Total Length of Fwd Packets,Label
0,0,128.0,128.000000,0.00000,2046,193460,164110.0,3.869200e+04,3.0,7.124800e+04,29304.0,35.0,35.00,35.0,0.000000,2,4,256.0,140.0,1
1,1,0.0,0.000000,0.00000,7,4016867,4011885.0,1.004217e+06,10.0,2.005113e+06,4016867.0,6.0,6.00,6.0,0.000000,0,5,0.0,30.0,0
2,2,0.0,0.000000,0.00000,2,5318063,5302748.0,1.772688e+06,97.0,3.057131e+06,5318063.0,6.0,4.00,0.0,3.464102,1,3,0.0,12.0,1
3,3,48.0,48.000000,0.00000,1081081,148,99.0,4.933333e+01,1.0,4.901360e+01,48.0,32.0,32.00,32.0,0.000000,2,2,96.0,64.0,1
4,4,1486.0,186.136364,426.14309,57,117320451,10100000.0,2.550445e+06,39.0,4.191832e+06,117000000.0,453.0,107.64,0.0,181.667939,22,25,4095.0,2691.0,1


In [5]:
newdf = pd.read_csv('/content/drive/MyDrive/Refined_data_set(label_encoding_7_features).csv')

In [6]:
newdf.shape

(1336498, 9)

In [7]:
newdf.head()

,Unnamed: 0,Bwd Packet Length Std,Flow Bytes/s,Total Length of Fwd Packets,Fwd Packet Length Std,Flow IAT Std,Flow IAT Min,Fwd IAT Total,Label
0,0,0.00000,2046,140.0,0.000000,7.124800e+04,3.0,29304.0,1
1,1,0.00000,7,30.0,0.000000,2.005113e+06,10.0,4016867.0,0
2,2,0.00000,2,12.0,3.464102,3.057131e+06,97.0,5318063.0,1
3,3,0.00000,1081081,64.0,0.000000,4.901360e+01,1.0,48.0,1
4,4,426.14309,57,2691.0,181.667939,4.191832e+06,39.0,117000000.0,1


In [8]:
newdf.drop('Unnamed: 0', axis=1, inplace=True)

In [9]:
newdf.head()

,Bwd Packet Length Std,Flow Bytes/s,Total Length of Fwd Packets,Fwd Packet Length Std,Flow IAT Std,Flow IAT Min,Fwd IAT Total,Label
0,0.00000,2046,140.0,0.000000,7.124800e+04,3.0,29304.0,1
1,0.00000,7,30.0,0.000000,2.005113e+06,10.0,4016867.0,0
2,0.00000,2,12.0,3.464102,3.057131e+06,97.0,5318063.0,1
3,0.00000,1081081,64.0,0.000000,4.901360e+01,1.0,48.0,1
4,426.14309,57,2691.0,181.667939,4.191832e+06,39.0,117000000.0,1


In [10]:
X=newdf[["Bwd Packet Length Std", "Flow Bytes/s", "Total Length of Fwd Packets", "Fwd Packet Length Std",
     "Flow IAT Std", "Flow IAT Min", "Fwd IAT Total"]]
y=newdf[["Label"]]

In [11]:
print(X.shape)
print(y.shape)

(1336498, 7)
(1336498, 1)


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [13]:
scaler = Normalizer().fit(X_train)
X_train = scaler.transform(X_train)

scaler = Normalizer().fit(y_train)
y_train = scaler.transform(y_train)


In [19]:
model = Sequential()
model.add(Dense(350,input_dim=7,activation='relu'))  
model.add(Dropout(0.10))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 350)               2800      
                                                                 
 dropout_1 (Dropout)         (None, 350)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 351       
                                                                 
 activation_1 (Activation)   (None, 1)                 0         
                                                                 
Total params: 3,151
Trainable params: 3,151
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=512, epochs=80)

Epoch 1/80
2089/2089 [==============================] - 8s 4ms/step - loss: 0.3811 - accuracy: 0.8372 - val_loss: 5846716.0000 - val_accuracy: 0.8240
Epoch 2/80
2089/2089 [==============================] - 6s 3ms/step - loss: 0.3519 - accuracy: 0.8542 - val_loss: 12675687.0000 - val_accuracy: 0.8240
Epoch 3/80
2089/2089 [==============================] - 6s 3ms/step - loss: 0.3479 - accuracy: 0.8547 - val_loss: 19020014.0000 - val_accuracy: 0.8241
Epoch 4/80
2089/2089 [==============================] - 7s 3ms/step - loss: 0.3448 - accuracy: 0.8554 - val_loss: 25165342.0000 - val_accuracy: 0.8241
Epoch 5/80
2089/2089 [==============================] - 6s 3ms/step - loss: 0.3422 - accuracy: 0.8557 - val_loss: 30317332.0000 - val_accuracy: 0.8241
Epoch 6/80
2089/2089 [==============================] - 7s 3ms/step - loss: 0.3397 - accuracy: 0.8561 - val_loss: 35826904.0000 - val_accuracy: 0.8241
Epoch 7/80
2089/2089 [==============================] - 7s 3ms/step - loss: 0.3370 - accuracy: 

In [21]:
from sklearn import metrics
def decode_sentiment(score):
    return 1 if score>0.5 else 0

In [22]:
scores = model.predict(X_test)
y_pred = [decode_sentiment(score) for score in scores]
accuracy=metrics.accuracy_score(y_test, y_pred)
print(f'Accuracy of DNN_1 is {accuracy:.2f}%')

Accuracy of DNN_1 is 0.82%


In [23]:
print("classification report:")
print(metrics.classification_report(y_test, y_pred))

classification report:
              precision    recall  f1-score   support

           0       0.22      0.01      0.02     47065
           1       0.82      0.99      0.90    220235

    accuracy                           0.82    267300
   macro avg       0.52      0.50      0.46    267300
weighted avg       0.72      0.82      0.75    267300

